In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import joblib

In [2]:
class RBM(nn.Module):
    def __init__(self, inp_dim, hid_dim, lr=0.01):
        super(RBM, self).__init__()
        self.hid_dim = hid_dim
        self.inp_dim = inp_dim
        
        self.W = nn.Parameter(torch.randn(inp_dim, hid_dim) * 0.1)
        self.bh = nn.Parameter(torch.zeros(hid_dim))
        self.bv = nn.Parameter(torch.zeros(inp_dim))
        
        self.lr = lr
        self.opt = optim.Adam(self.parameters(), lr=lr)
    
    def sample_hid(self, v):
        h_prob = torch.sigmoid(torch.matmul(v, self.W) + self.bh)
        return (torch.rand_like(h_prob) < h_prob).float()
    
    def sample_vis(self, h):
        v_prob = torch.sigmoid(torch.matmul(h, self.W.t()) + self.bv)
        return (torch.rand_like(v_prob) < v_prob).float()
    
    def contrastive_divergence(self, v0):
        h0 = self.sample_hid(v0)
        v1 = self.sample_vis(h0)
        h1 = self.sample_hid(v1)
        
        pos_grad = torch.matmul(v0.t(), h0)
        neg_grad = torch.matmul(v1.t(), h1)
        
        self.W.data += self.lr * (pos_grad - neg_grad) / v0.size(0)
        self.bh.data += self.lr * torch.mean(h0 - h1, dim=0)
        self.bv.data += self.lr * torch.mean(v0 - v1, dim=0)
    
    def train_rbm(self, data, epochs=10, batch_size=64, device='cuda'):
        dataset = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)
        self.to(device)
        
        for epoch in range(epochs):
            for batch in dataset:
                batch = batch.to(device)
                self.contrastive_divergence(batch)
            print(f"Epoch {epoch+1}/{epochs} completed.")
    
    def transform(self, data):
        return torch.sigmoid(torch.matmul(data, self.W) + self.bh)

In [3]:
train = np.loadtxt('../DATA/train.csv', delimiter=',', skiprows=1)  # Adjust path if needed
train = np.nan_to_num(train)
scaler = MinMaxScaler()
train = scaler.fit_transform(train)
joblib.dump(scaler, 'scaler.pkl')

ValueError: could not convert string 'Benign' to float64 at row 0, column 18.

In [ ]:
device = 'mps' if torch.backends.mps.is_available() else 'cpu'
x = torch.tensor(train, dtype=torch.float32).to(device)


In [ ]:
inp = x.shape[1]
hid = 128  
rbm = RBM(inp, hid, lr=0.01)
rbm.train_rbm(x, epochs=50, batch_size=32, device=device)